In [78]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from pyquery import PyQuery as pq
from pymongo import MongoClient


In [156]:
def init_browser(key_word):
    """初始化浏览器，转到相应的页面，并返回页面最大页码数"""
    # 初始化浏览器
    browser = webdriver.Chrome()
    # 设置显示等待时间
    wait = WebDriverWait(browser, 10)
    # 跳转到京东
    browser.get('https://www.jd.com/')
    # 获取输入框，并输入搜索内容
    search_input = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#key'))
    )
    search_input.clear()
    # 回车，搜索
    search_input.send_keys(key_word)
    search_input.send_keys(Keys.ENTER)
    time.sleep(2)
    # 翻到网页最下面
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    # 执行js，获取最大页码数
    time.sleep(2)
    max_page = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#J_bottomPage .p-skip em > b'))
    )
    print('初始化浏览器，并获取最大页码数成功：max_page:{}'.format(max_page.text))
    return browser, wait, int(max_page.text)


def get_page_source(browser, wait):
    """获取网页源代码"""
    try:
        # 让滚动条滚到最下面获取所有的li
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        time.sleep(2)
        wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#J_goodsList ul'))
        )
        print('获取网页源代码成功')
        page_source = browser.page_source
#         print(page_source)
        return page_source
    except TimeoutException as e:
        print('Time out')
        get_page_source()
    except Exception as e:
        print("Error is {}".format(e))

def parse_url(page_source):
    """解析网页，返回相关商品信息"""
    print('开始解析')
    doc = pq(page_source)
    items = doc('#J_goodsList ul li').items()
    product_list = []
    items = doc('#J_goodsList .gl-warp .gl-item').items()
    for item in items:
        product_data = {}
        price = item('.p-price strong i').text()
#         print(price)
        product_data['href'] = "https:" + item('.p-name a').attr('href')
        product_data['title'] = item('.p-name a').attr('title')  # 获取商品描述
        product_data['price'] = float(price)
        product_data['comment_count'] = item('.p-commit strong ').text()
        product_data['shop'] = item('.p-shop .J_im_icon a').text()
        product_list.append(product_data)
    print('解析成功')
#     print(product_list)
    return product_list

def start_mongo():
    client = MongoClient(host='localhost', port=27017)
    collection = client['jd']['jd_product_watch']
    
    return collection

def save_to_database(product_list, collection):
    for product_data in product_list:
        try:
            collection.insert_one(product_data)
        except Exception as e:
            print('error is {}'.format(e))
    print('保存成功')
    
def turn_page(browser, wait, page):
    page_input = browser.find_element_by_css_selector('#J_bottomPage .input-txt')
    submit_button = browser.find_element_by_css_selector('#J_bottomPage .btn')
    page_input.clear()
    page_input.send_keys(page)
    print('跳转到 {}页'.format(page))
    submit_button.click()
        
def main(browser, wait, page, collection):
    """实现主程序"""
    # 1.翻到网页最下面，让所有的内容加载完成，并返回网页源码
    print('开始第{}页'.format(page-1))
    page_source = get_page_source(browser, wait)
    # 2.解析网页源码，返回解析后的商品信息
    product_list = parse_url(page_source)
    # 3.保存到数据库
    save_to_database(product_list, collection)
    print('结束第{}页'.format(page-1))
    # 4.翻页（如果当前页码是最大页码数则不翻页）
    if page < max_page:
        turn_page(browser, wait, page)
        

In [157]:
if __name__ == "__main__":
    key_word = "手表"
    # 1.初始化浏览器，转到相应的页面，并返回页面最大页码数
    browser, wait, max_page = init_browser(key_word)
    collection = start_mongo()
    try:
        for page in range(2, max_page+2):
    #     for page in range(95,98):
            main(browser,wait, page, collection)
    except Exception as e:
        print('Error is : {}'.format(e))
    finally:
        browser.quit()

初始化浏览器，并获取最大页码数成功：max_page:100
开始第1页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第1页
跳转到 2页
开始第2页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第2页
跳转到 3页
开始第3页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第3页
跳转到 4页
开始第4页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第4页
跳转到 5页
开始第5页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第5页
跳转到 6页
开始第6页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第6页
跳转到 7页
开始第7页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第7页
跳转到 8页
开始第8页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第8页
跳转到 9页
开始第9页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第9页
跳转到 10页
开始第10页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第10页
跳转到 11页
开始第11页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第11页
跳转到 12页
开始第12页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第12页
跳转到 13页
开始第13页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第13页
跳转到 14页
开始第14页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第14页
跳转到 15页
开始第15页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第15页
跳转到 16页
开始第16页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第16页
跳转到 17页
开始第17页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第17页
跳转到 18页
开始第18页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第18页
跳转到 19页
开始第19页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第19页
跳转到 20页
开始第20页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第20页
跳转到 21页
开始第21页
获取网页源代码成功
开始解析
解析成功
保存成功
结束第21页
跳转到 22页
开始第22页
获

In [126]:
client = MongoClient(host='localhost', port=27017)
collection = client['jd']['jd_product3']


In [127]:
collection.create_index([('name', 1), ('unique', 1), ('dropDups', 1)])

'name_1_unique_1_dropDups_1'